In [ ]:
import sqlite3
from getpass import getpass  # For secure PIN input

class MiniATMSimulator:
    def __init__(self):
        self.conn = sqlite3.connect('atm_database.db')
        self.create_tables()
        self.current_account = None
        self.is_locked = False

    def create_tables(self):
        cursor = self.conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS accounts (
                account_number TEXT PRIMARY KEY,
                name TEXT NOT NULL,
                pin TEXT NOT NULL,
                balance REAL DEFAULT 1000.00,
                is_locked INTEGER DEFAULT 0
            )
        ''')
        self.conn.commit()

        # Create a sample account if none exists
        cursor.execute("SELECT COUNT(*) FROM accounts")
        if cursor.fetchone()[0] == 0:
            cursor.execute('''
                INSERT INTO accounts (account_number, name, pin, balance)
                VALUES ('1001001234', 'John Doe', '1234', 1000.00)
            ''')
            self.conn.commit()

    def run(self):
        print("\n=== Welcome to Mini ATM Simulator ===")

        account_number = input("Enter your account number: ")
        if not self.verify_account(account_number):
            return

        # PIN verification
        attempts = 3
        while attempts > 0 and not self.is_locked:
            entered_pin = getpass("Enter your 4-digit PIN: ")  # Hides PIN input
            if self.verify_pin(account_number, entered_pin):
                self.current_account = account_number
                self.show_menu()
                return
            else:
                attempts -= 1
                if attempts > 0:
                    print(f"\nWRONG PIN. ACCESS DENIED. {attempts} attempts remaining.")
                else:
                    print("\nWRONG PIN. ACCESS DENIED.")
                    print("Too many incorrect attempts. Card blocked.")
                    self.lock_account(account_number)
                    self.is_locked = True

    def verify_account(self, account_number):
        cursor = self.conn.cursor()
        cursor.execute("SELECT is_locked FROM accounts WHERE account_number = ?", (account_number,))
        result = cursor.fetchone()

        if not result:
            print("Account not found.")
            return False
        elif result[0] == 1:
            print("This account is locked. Please contact your bank.")
            return False
        return True

    def verify_pin(self, account_number, pin):
        cursor = self.conn.cursor()
        cursor.execute("SELECT pin FROM accounts WHERE account_number = ?", (account_number,))
        result = cursor.fetchone()
        return result and result[0] == pin

    def lock_account(self, account_number):
        cursor = self.conn.cursor()
        cursor.execute("UPDATE accounts SET is_locked = 1 WHERE account_number = ?", (account_number,))
        self.conn.commit()

    def show_menu(self):
        while True:
            print("\n=== Main Menu ===")
            print("1. Check Balance")
            print("2. Withdraw Cash")
            print("3. Deposit Cash")
            print("4. Change PIN")
            print("5. View Account Info")
            print("6. Exit")

            choice = input("Enter your choice (1-6): ")

            if choice == "1":
                self.check_balance()
            elif choice == "2":
                self.withdraw_cash()
            elif choice == "3":
                self.deposit_cash()
            elif choice == "4":
                self.change_pin()
            elif choice == "5":
                self.view_account_info()
            elif choice == "6":
                print("\nThank you for using Mini ATM. Goodbye!")
                break
            else:
                print("Invalid choice. Please try again.")

    def check_balance(self):
        cursor = self.conn.cursor()
        cursor.execute("SELECT balance FROM accounts WHERE account_number = ?", (self.current_account,))
        balance = cursor.fetchone()[0]
        print(f"\nYour current balance is: ${balance:.2f}")

    def withdraw_cash(self):
        try:
            amount = float(input("\nEnter amount to withdraw: $"))
            cursor = self.conn.cursor()

            # Check balance first
            cursor.execute("SELECT balance FROM accounts WHERE account_number = ?", (self.current_account,))
            balance = cursor.fetchone()[0]

            if amount > balance:
                print("Insufficient funds.")
            elif amount <= 0:
                print("Invalid amount.")
            else:
                # Update balance
                cursor.execute("""
                    UPDATE accounts
                    SET balance = balance - ?
                    WHERE account_number = ?
                """, (amount, self.current_account))
                self.conn.commit()
                print(f"Withdrawal successful. Remaining balance: ${balance - amount:.2f}")
        except ValueError:
            print("Invalid input. Please enter a numeric value.")

    def deposit_cash(self):
        try:
            amount = float(input("\nEnter amount to deposit: $"))
            if amount <= 0:
                print("Invalid amount.")
            else:
                cursor = self.conn.cursor()
                cursor.execute("""
                    UPDATE accounts
                    SET balance = balance + ?
                    WHERE account_number = ?
                """, (amount, self.current_account))
                self.conn.commit()

                # Get updated balance
                cursor.execute("SELECT balance FROM accounts WHERE account_number = ?", (self.current_account,))
                new_balance = cursor.fetchone()[0]
                print(f"Deposit successful. New balance: ${new_balance:.2f}")
        except ValueError:
            print("Invalid input. Please enter a numeric value.")

    def change_pin(self):
        old_pin = getpass("\nEnter current PIN: ")
        if not self.verify_pin(self.current_account, old_pin):
            print("Incorrect PIN.")
            return

        new_pin = getpass("Enter new 4-digit PIN: ")
        if len(new_pin) != 4 or not new_pin.isdigit():
            print("PIN must be 4 digits.")
            return

        confirm_pin = getpass("Confirm new PIN: ")
        if new_pin == confirm_pin:
            cursor = self.conn.cursor()
            cursor.execute("""
                UPDATE accounts
                SET pin = ?
                WHERE account_number = ?
            """, (new_pin, self.current_account))
            self.conn.commit()
            print("PIN changed successfully.")
        else:
            print("PINs do not match.")

    def view_account_info(self):
        cursor = self.conn.cursor()
        cursor.execute("""
            SELECT account_number, name, balance
            FROM accounts
            WHERE account_number = ?
        """, (self.current_account,))
        account_number, name, balance = cursor.fetchone()

        print("\n=== Account Information ===")
        print(f"Account Number: {account_number}")
        print(f"Account Holder: {name}")
        print(f"Current Balance: ${balance:.2f}")

    def __del__(self):
        self.conn.close()

# Run the ATM simulator
if __name__ == "__main__":
    atm = MiniATMSimulator()
    atm.run()


=== Welcome to Mini ATM Simulator ===


Exception ignored in: <function MiniATMSimulator.__del__ at 0x7a12670c32e0>
Traceback (most recent call last):
  File "<ipython-input-2-ab700134b7b6>", line 201, in __del__
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 134220035031040 and this is thread id 134219414521408.



WRONG PIN. ACCESS DENIED. 2 attempts remaining.

=== Main Menu ===
1. Check Balance
2. Withdraw Cash
3. Deposit Cash
4. Change PIN
5. View Account Info
6. Exit

Your current balance is: $1000500.00

=== Main Menu ===
1. Check Balance
2. Withdraw Cash
3. Deposit Cash
4. Change PIN
5. View Account Info
6. Exit
